In [1]:
from OSLPP import *
from DEEPNN import get_simple_deep_nn, fix_batchnorms
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import os
import pandas as pd
from torch.optim.lr_scheduler import StepLR
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [4]:
def get_abbrev(source, target): return source[0].upper() + target[0].upper()
def fmeasures(d): return ' '.join([f'{k}={v*100:.2f}' for (k,v) in d.items()])

In [5]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, i): return self.features[i][0], self.labels[i]

In [6]:
def _create_balanced_sampler(ds:FeaturesDataset):
    freq2 = Counter(ds.labels.tolist())
    class_weight = {x: 1.0 / freq2[x] for x in freq2}
    source_weights = [class_weight[x] for x in ds.labels.tolist()]
    sampler = torch.utils.data.WeightedRandomSampler(source_weights, len(ds.labels.tolist()))
    return sampler

def _create_trn_dataloader(ds, batch_size, balanced):
    if balanced: return torch.utils.data.DataLoader(ds, batch_size=batch_size, sampler=_create_balanced_sampler(ds), drop_last=True)
    else: torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)

In [7]:
def get_model(n_classes):
    return get_simple_deep_nn(n_classes)

In [8]:
def train_initial_NN(feats_S, lbls_S, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    assert (lbls_S.unique() == torch.arange(num_src_classes)).all()
    model = get_model(num_src_classes).cuda().train()
    model.apply(fix_batchnorms)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats_S, lbls_S)
    dl = _create_trn_dataloader(ds, 32, balanced)
    print('CLASSES:', num_src_classes)
    for ep in (pbar := tqdm(range(num_epochs))):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            preds = model(f.cuda())
            loss = loss_fn(preds, l.cuda())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        pbar.set_description('Loss: ' + str(avg_loss / len(dl)))
    return model.eval()

In [9]:
def entropy_loss(logits):
    probs = F.softmax(logits, dim=1)
    entropy =  - probs * probs.log()
    entropy = entropy.sum(dim=1)
    return entropy.mean()

In [10]:
def train_NN(feats, lbls, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    feats, lbls = Subset(feats, np.arange(0, len(lbls))[lbls >= 0]), lbls[lbls >= 0]
    feats, lbls = Subset(feats, np.arange(0, len(lbls))[lbls < num_src_classes]), lbls[lbls < num_src_classes]
    model = get_model(num_src_classes).cuda().train()
    model.apply(fix_batchnorms)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats, lbls)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in (pbar := tqdm(range(num_epochs))):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        pbar.set_description('Loss: ' + str(avg_loss / len(dl)))
    return model.eval()

In [11]:
def predict_NN(model, feats, lbls):
    ds = FeaturesDataset(feats, lbls)
    dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for f,l in dl:
            out = model(f.cuda())
            preds.append(F.softmax(out, dim=1).detach().cpu())
    preds = torch.cat(preds, dim=0)
    return preds

In [12]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [13]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [14]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [15]:
# sources = ['train'] 
# targets = ['validation']

In [16]:
def set_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
results = pd.DataFrame({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
# select_reject_mode = SelectRejectMode.ENTROPY
select_reject_mode = SelectRejectMode.CONFIDENCE
# select_reject_mode = SelectRejectMode.MARGIN
# for source in sources:
#     for target in targets:
#         if source == target:
#             continue
#         for lr in [1e-3, 1e-4, 5e-4]:
#             for n_r in [0.15, 0.25, 0.35, 0.45]:
for (source, target), (common, tgt_private) in config.items():
    for epochs in [35]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
                for seed in range(1):
                    set_seed(seed)
                    print(source, '->', target, 'lr=', lr, 'seed=', seed)
                    params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                              dataset='DomainNet_DCC', source=source, target=target,
                              num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                    (feats_S, lbls_S), (feats_T, lbls_T) = create_datasets_sub(params.dataset, 
                                                                               params.source, 
                                                                               params.target, 
                                                                               common, 
                                                                               tgt_private,
                                                                               is_images=True)
                    params.n_r = int(len(lbls_T) * n_r)
                    num_src_classes = params.num_common + params.num_src_priv

                    # initial
                    lbls_S, lbls_T = torch.tensor(lbls_S), torch.tensor(lbls_T)
                    feats_all = ConcatDataset([feats_S, feats_T])
                    pseudo_labels = -torch.ones_like(lbls_T)
                    rejected = torch.zeros_like(pseudo_labels)

                    t = 1
                    model = train_initial_NN(feats_S, lbls_S, epochs, params, balanced=True, lr=lr)
                    preds_S_2 = predict_NN(model, feats_S, lbls_S)
                    preds_T_2 = predict_NN(model, feats_T, lbls_T)
                    feats_all = ConcatDataset([feats_S, feats_T])

                    if select_reject_mode == SelectRejectMode.CONFIDENCE:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected(confs, params.n_r)
                    elif select_reject_mode == SelectRejectMode.ENTROPY:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected_by_entropy(confs, params.n_r, preds_T_2)
                    elif select_reject_mode == SelectRejectMode.MARGIN:
                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected_by_margin(confs, params.n_r, preds_T_2)
                    else:
                        raise Exception('Incorrect mode.')

                    rejected = torch.tensor(rejected)
                    selected = selected * (1 - rejected)

                    pseudo_labels = cs_pseudo_labels.clone()
                    pseudo_labels[rejected == 1] = num_src_classes
                    pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, 
                            lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                    where = torch.where((selected == 1) + (rejected == 1))[0]
                    metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                    print('______')
                    print(f'Iteration t={t}')
                    print('all: ', fmeasures(metrics))
                    print('selected: ', fmeasures(metrics_selected))
                    
                    for t in range(2, params.T):
                        print('TRAIN NN:', t)
                        model = train_NN(feats_all, torch.cat((lbls_S, pseudo_labels), axis=0), epochs, params, balanced=True, lr=lr)
                        preds_S_2 = predict_NN(model, feats_S, lbls_S)
                        preds_T_2 = predict_NN(model, feats_T, lbls_T)

                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = torch.tensor(select_closed_set_pseudo_labels(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, select_reject_mode))
                        selected = selected * (1 - rejected)

                        rejected_new = (confs < min(confs[rejected == 1].max().item(), confs[selected == 1].min().item())).int()

                        rejected[rejected_new == 1] = 1
                        selected = selected * (1 - rejected)

                        pseudo_labels = cs_pseudo_labels.clone()
                        pseudo_labels[rejected == 1] = num_src_classes
                        pseudo_labels[(rejected == 0) * (selected == 0)] = -1
                        
                        print('Rejected:', len(rejected[rejected == 1]), '/', len(rejected))
                        print('Selected:', len(selected[selected == 1]), '/', len(selected))

                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                        where = torch.where((selected == 1) + (rejected == 1))[0]
                        metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                        print('______')
                        print(f'Iteration t={t}')
                        print('all: ', fmeasures(metrics))
                        print('selected: ', fmeasures(metrics_selected))

                    print((pseudo_labels == -1).sum())

                    _rejected = rejected
                    metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), _rejected.numpy())
                    print('all: ', fmeasures(metrics))

                    results = results.append({'source': source, 'target': target, 'desc': fmeasures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs}, ignore_index=True)

painting -> sketch lr= 0.001 seed= 0
{24: 0, 87: 1, 94: 2, 106: 3, 116: 4, 121: 5, 131: 6, 167: 7, 169: 8, 171: 9, 247: 10, 258: 11, 268: 12, 269: 13, 272: 14, 280: 15, 281: 16, 299: 17, 311: 18, 322: 19, 29: 20, 108: 21, 236: 22, 252: 23, 256: 24, 264: 25, 266: 26, 275: 27, 276: 28, 278: 29}
CLASSES: 20


Loss: 2.9958654892060066: 100%|█████████████████| 35/35 [43:33<00:00, 74.68s/it]


Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [15 15 15 ... 15 15 15]
Labels: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 20 20 20 20 20 20  1  1  1  1
  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3 20 20 20 20 20 20  4  4  4  4  4
  4  5  5  5  5  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  8  8  8  8
  9  9  9  9  9  9  9  9 20 20 20 10 10 10 10 10 10 10 10 10 20 20 20 20
 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
 20 20 20 20 20 20 20 20 20 20 20 20 20 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 20 20 20 20 20 20 20 20 20 20 20 20 20 20
 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20
 20 20 20 20 20 20 20 20 20 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13 13 13 

Loss: 3.0058119261442724: 100%|█████████████████| 35/35 [47:57<00:00, 82.22s/it]


Rejected: 1221 / 2755
Selected: 615 / 2755
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [16 16 16 ... 16 16 16]
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [16 16 16 ... 16 16 16]
______
Iteration t=2
all:  cs/acc_i=11.83 cs/acc_c=5.00 os/recall_knw=57.91 os/recall_unk=48.06 total/acc_i=23.19 total/acc_c=5.65 total/h_score=6.58
selected:  cs/acc_i=10.44 cs/acc_c=5.00 os/recall_knw=35.75 os/recall_unk=70.11 total/acc_i=30.12 total/acc_c=5.68 total/h_score=4.75
TRAIN NN: 3


...

Loss: 2.995461622318188: 100%|██████████████████| 35/35 [43:19<00:00, 74.28s/it]


Rejected: 1796 / 2755
Selected: 959 / 2755
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [0 0 0 ... 0 0 0]
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [0 0 0 ... 0 0 0]
______
Iteration t=6
all:  cs/acc_i=3.65 cs/acc_c=5.00 os/recall_knw=37.57 os/recall_unk=69.81 total/acc_i=27.15 total/acc_c=5.52 total/h_score=4.46
selected:  cs/acc_i=3.65 cs/acc_c=5.00 os/recall_knw=37.57 os/recall_unk=69.81 total/acc_i=27.15 total/acc_c=5.52 total/h_score=4.46
TRAIN NN: 7


Loss: 0.37736009840260853: 100%|████████████████| 35/35 [51:09<00:00, 87.69s/it]


Rejected: 1808 / 2755
Selected: 805 / 2755
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [ 0 14  0 ...  7  0  0]
Labels: [ 0  0  0 ... 19 19 19]
Pseudo: [ 0 14  0 ...  7  0  0]
______
Iteration t=7
all:  cs/acc_i=7.13 cs/acc_c=7.30 os/recall_knw=36.99 os/recall_unk=70.00 total/acc_i=27.84 total/acc_c=5.90 total/h_score=5.19
selected:  cs/acc_i=7.03 cs/acc_c=7.30 os/recall_knw=33.52 os/recall_unk=73.72 total/acc_i=29.05 total/acc_c=5.88 total/h_score=4.82
TRAIN NN: 8


Loss: 1.2404219840196955:  63%|██████████▋      | 22/35 [32:44<19:19, 89.16s/it]

In [ ]:
results.to_csv('results/abacaba-deep.csv', header=True, index=False)

In [20]:
# df = pd.read_csv('office_home_oslpp_nn_raw.csv')
# df['h_score'] = df['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))
# import matplotlib.pyplot as plt
# for n, gr in df.groupby(['source', 'target']):
#     stat = gr.groupby(['lr', 'n_r'])['h_score'].mean()
#     exps = np.array(list(stat.index))
#     fig, ax = plt.subplots()
#     ax.scatter(exps[:, 0], exps[:, 1], c=stat.values)
#     for i, txt in enumerate(stat.values):
#         ax.annotate(round(txt, 2), (exps[i][0], exps[i][1]))
#     plt.title(n)
#     plt.xlabel('lr')
#     plt.ylabel('n_r')
#     plt.xticks(exps[:, 0])
#     plt.yticks(exps[:, 1])
#     fig.colorbar(ax.get_children()[0])
#     plt.show()

In [20]:
results

,source,target,desc,lr,seed,epochs,n_r
0,painting,sketch,cs/acc_i=52.60 cs/acc_c=54.16 os/recall_knw=55...,0.0010,0.0,10.0,0.10
1,painting,sketch,cs/acc_i=52.18 cs/acc_c=53.89 os/recall_knw=55...,0.0010,1.0,10.0,0.10
2,painting,sketch,cs/acc_i=52.11 cs/acc_c=53.66 os/recall_knw=55...,0.0010,2.0,10.0,0.10
3,painting,sketch,cs/acc_i=51.95 cs/acc_c=53.61 os/recall_knw=55...,0.0010,3.0,10.0,0.10
4,painting,sketch,cs/acc_i=52.19 cs/acc_c=53.80 os/recall_knw=58...,0.0010,4.0,10.0,0.10
...,...,...,...,...,...,...,...
715,sketch,painting,cs/acc_i=73.66 cs/acc_c=74.10 os/recall_knw=66...,0.0001,5.0,50.0,0.35
716,sketch,painting,cs/acc_i=73.39 cs/acc_c=73.77 os/recall_knw=67...,0.0001,6.0,50.0,0.35
717,sketch,painting,cs/acc_i=72.88 cs/acc_c=73.25 os/recall_knw=68...,0.0001,7.0,50.0,0.35
718,sketch,painting,cs/acc_i=73.64 cs/acc_c=74.03 os/recall_knw=67...,0.0001,8.0,50.0,0.35


In [30]:
results['epoch'] = pd.Series(results.index).apply(lambda i: epochs[(i // 9) % 3])

In [21]:
wd = results.copy()
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [22]:
wd

,source,target,desc,lr,seed,epochs,n_r,h_score
0,painting,sketch,cs/acc_i=52.60 cs/acc_c=54.16 os/recall_knw=55...,0.0010,0.0,10.0,0.10,50.79
1,painting,sketch,cs/acc_i=52.18 cs/acc_c=53.89 os/recall_knw=55...,0.0010,1.0,10.0,0.10,50.06
2,painting,sketch,cs/acc_i=52.11 cs/acc_c=53.66 os/recall_knw=55...,0.0010,2.0,10.0,0.10,49.67
3,painting,sketch,cs/acc_i=51.95 cs/acc_c=53.61 os/recall_knw=55...,0.0010,3.0,10.0,0.10,50.35
4,painting,sketch,cs/acc_i=52.19 cs/acc_c=53.80 os/recall_knw=58...,0.0010,4.0,10.0,0.10,50.92
...,...,...,...,...,...,...,...,...
715,sketch,painting,cs/acc_i=73.66 cs/acc_c=74.10 os/recall_knw=66...,0.0001,5.0,50.0,0.35,65.86
716,sketch,painting,cs/acc_i=73.39 cs/acc_c=73.77 os/recall_knw=67...,0.0001,6.0,50.0,0.35,65.38
717,sketch,painting,cs/acc_i=72.88 cs/acc_c=73.25 os/recall_knw=68...,0.0001,7.0,50.0,0.35,65.47
718,sketch,painting,cs/acc_i=73.64 cs/acc_c=74.03 os/recall_knw=67...,0.0001,8.0,50.0,0.35,65.22


In [23]:
wd.groupby(['source', 'target', 'lr', 'n_r', 'epochs'])['h_score'].mean().to_frame(name = 'h_score').reset_index().sort_values(by='h_score', ascending=False).drop_duplicates(['source', 'target'])

,source,target,lr,n_r,epochs,h_score
0,painting,real,0.0001,0.10,10.0,81.672
61,sketch,real,0.0001,0.10,50.0,80.493
24,real,painting,0.0001,0.10,10.0,66.994
49,sketch,painting,0.0001,0.10,50.0,66.113
37,real,sketch,0.0001,0.10,50.0,57.341
16,painting,sketch,0.0001,0.35,10.0,52.816


In [26]:
wd['name'] = wd['source'].apply(lambda x: x[0].upper()) + wd['target'].apply(lambda x: x[0].upper())

In [39]:
wd.pivot_table(index=['n_r', 'lr', 'epochs'], columns='name', values='h_score', aggfunc=[lambda x: round(np.mean(x), 2)]).sort_index(key=lambda x: x, ascending=False)

<lambda>                                   
name                     PR     PS     RP     RS     SP     SR
n_r  lr     epochs                                            
0.35 0.0010 50.0      72.80  45.50  60.30  50.75  52.58  72.25
            10.0      77.33  49.99  62.89  55.10  63.92  74.75
     0.0001 50.0      78.81  51.48  65.33  57.14  65.35  74.63
            10.0      79.42  52.82  65.99  56.25  56.49  68.81
0.25 0.0010 50.0      73.04  45.23  60.33  50.67  52.53  72.33
            10.0      77.09  50.20  63.08  55.03  63.92  76.93
     0.0001 50.0      79.63  51.32  65.79  57.30  65.69  75.52
            10.0      79.68  52.74  66.89  56.28  54.33  69.98
0.10 0.0010 50.0      73.26  45.78  61.24  50.73  52.67  72.69
            10.0      78.13  50.25  63.18  55.79  63.70  79.03
     0.0001 50.0      80.01  51.29  65.41  57.34  66.11  80.49
            10.0      81.67  52.49  66.99  56.16  51.87  71.82

In [ ]:
wd.set_index(['lr', 'epoch', 'n_r']).head(30)

In [44]:
wd = pd.read_csv('results/dcc__entropy__StepLR__50_epochs__nn_raw.csv')

In [45]:
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [46]:
wd

,source,target,desc,lr,seed,n_r,h_score
0,painting_train,real_train,cs/acc_i=59.98 cs/acc_c=60.09 os/recall_knw=71...,0.0005,0.0,0.15,54.40
1,painting_train,sketch_train,cs/acc_i=26.40 cs/acc_c=25.41 os/recall_knw=53...,0.0005,0.0,0.15,30.42
2,real_train,painting_train,cs/acc_i=39.74 cs/acc_c=39.39 os/recall_knw=65...,0.0005,0.0,0.15,40.83
3,real_train,sketch_train,cs/acc_i=24.48 cs/acc_c=23.68 os/recall_knw=51...,0.0005,0.0,0.15,29.29
4,sketch_train,painting_train,cs/acc_i=37.51 cs/acc_c=34.45 os/recall_knw=70...,0.0005,0.0,0.15,35.06
5,sketch_train,real_train,cs/acc_i=56.05 cs/acc_c=57.00 os/recall_knw=71...,0.0005,0.0,0.15,48.56
